In [ ]:
from __future__ import print_function

import pandas as pd
import numpy as np

from src import tracker
from genepy import terra
from genepy.utils import helper as h
from genepy.google import gcp
from genepy import sequencing as seq
from gsheets import Sheets
from taigapy import TaigaClient
import dalmatian as dm
from genepy.google.google_sheet import dfToSheet

from IPython.display import Image,display
from bokeh.plotting import *

%load_ext autoreload
%autoreload 2
%load_ext rpy2.ipython
tc = TaigaClient()
output_notebook()

my_id = '~/.client_secret.json'
mystorage_id = "~/.storage.json"
# do the first steps of https://medium.com/craftsmenltd/from-csv-to-google-sheet-using-python-ef097cb014f9
creds = '../.credentials.json'

sheets = Sheets.from_files(my_id, mystorage_id)
replace = {'T': 'Tumor', 'N': 'Normal', 'm': 'Unknown', 'L': 'Unknown'}

## boot up

we are instanciating all the parameters needed for this pipeline to run

In [ ]:
samplesetname = "21Q1"

workspace1="terra-broad-cancer-prod/DepMap_WGS"
workspace2="terra-broad-cancer-prod/Getz_IBM_CellLines_WGS"

refworkspace="broad-firecloud-ccle/DepMap_WGS_CN"
cgaworkspace="broad-firecloud-ccle/DepMap_Mutation_Calling_CGA_pipeline-wgs"

source1="ccle"
source2="ibm"

refsheet_url = "https://docs.google.com/spreadsheets/d/1Pgb5fIClGnErEqzxpU7qqX6ULpGTDjvzWwDN8XUJKIY"
sheeturl = "https://docs.google.com/spreadsheets/d/115TUgA1t_mD32SnWAGpW9OKmJ2W5WYAOs3SuSdedpX4"
potential_list_url = "https://docs.google.com/spreadsheets/d/1BEgH03V4OmGhYeciLCZV00h6hp3WkO0basahS93akCE"

depmap_pv = "https://docs.google.com/spreadsheets/d/1uqCOos-T9EMQU7y2ZUw4Nm84opU5fIT1y7jet1vnScE"

release = samplesetname

WGSmethods = [
    "gatk/PreProcessingForVariantDiscovery_GATK4/8",
    "GP-TAG/Manta_SomaticSV/9",
    "gkugener/ArrayOfFilesToTxt/1",
    "vdauwera/BamToUnmappedRGBams/4",
    "gatk/CNV_Somatic_Pair_Workflow/9",
    "gkugener/Aggregate_CN_seg_files/2"
]

maxage='2020-09-10'

# version 102
ensemblserver = "http://nov2020.archive.ensembl.org/biomart" 

In [ ]:
wm1 = dm.WorkspaceManager(workspace1)
wm2 = dm.WorkspaceManager(workspace2)

refwm = dm.WorkspaceManager(refworkspace).disable_hound()
cgawm = dm.WorkspaceManager(cgaworkspace).disable_hound()

In [ ]:
extract_to_change = {'from_arxspan_id': 'participant'}
potential_list = sheets.get(potential_list_url).sheets[0].to_frame().values.T[0].tolist()
ccle_refsamples = sheets.get(refsheet_url).sheets[0].to_frame(index_col=0)
#ccle_refsamples=pd.read_csv('temp/updated_ref_samples.csv',index_col=0)

depmap_pv = sheets.get(depmap_pv).sheets[0].to_frame(header=2)
depmap_pv = depmap_pv.drop(depmap_pv.iloc[:1].index).set_index('DepMap_ID', drop=True)[['Culture Type', 'Culture Medium']]
depmap_master = tc.get(name='arxspan-cell-line-export-f808', file='ACH').set_index('arxspan_id',drop=True)
depmap_master = depmap_master.join(depmap_pv)

In [ ]:
ccle_refsamples = sheets.get(refsheet_url).sheets[0].to_frame(index_col=0)


In [ ]:
wm = dm.WorkspaceManager("broad-firecloud-ccle/DepMap_hg38_RNAseq")
a=ccle_refsamples.loc[["CDS-G0F5f5", "CDS-kU30H5", "CDS-fk564T"], ["stripped_cell_name", "participant_id", "cellosaurus_id", "internal_bam_filepath", "internal_bai_filepath", "legacy_bam_filepath", "legacy_bai_filepath", "parent_cell_line", "sex", "matched_normal", "age", "primary_site", "primary_disease", "subtype", "subsubtype", "origin", "mediatype", "condition", "baits", "source", "sequencing_date", 'size', 'crc32c_hash', 'md5_hash', 'legacy_size', "legacy_crc32c_hash"]]
a.index.name="sample_id"
a = a.rename(columns={'patient_id': 'participant_id'})
wm.upload_samples(a)

In [ ]:
wm.update_sample_set('toremap', a.index.tolist())

In [ ]:
wm.create_submission('star_v1-0_BETA_cfg', "toremap", "sample_set", "this.samples")

In [ ]:
sam= wm.get_samples()

In [ ]:
sam.loc[["CDS-G0F5f5", "CDS-kU30H5", "CDS-fk564T"], ['star_bam_file', 'star_bam_index']]

In [ ]:
a=terra.changeToBucket(sam.loc[["CDS-G0F5f5", "CDS-kU30H5", "CDS-fk564T"], ['star_bam_file', 'star_bam_index']],'gs://cclebams/rnasq_hg38/', name_col= "index" , values=['star_bam_file','star_bam_index'], filetypes=['bam', 'bai'], catchdup=True, test=False)

In [ ]:
ccle_refsamples.loc[["CDS-G0F5f5", "CDS-kU30H5", "CDS-fk564T"], ['internal_bam_filepath', 'internal_bai_filepath']]

In [ ]:
dfToSheet(ccle_refsamples,'ccle sample tracker', secret=creds)

## PART 2

In [ ]:
a = ! gsutil ls gs://cclebams/rnasq_hg38/

In [ ]:
set(ccle_refsamples[ccle_refsamples.datatype.isin(['rna'])].index) - set(["CDS"+i.split('/CDS')[-1].split('.Ali')[0] for i in a])

In [ ]:
l

In [ ]:
i  = ! gsutil ls gs://cclebams/**

In [ ]:
b = [a for a in i if a.endswith('.bam')]

In [ ]:
len(b)

In [ ]:
bi = [a for a in i if a.endswith('.bam') and a[:-4]+".bai" not in i and a+".bai" not in i]

In [ ]:
len(bi)

In [ ]:
! gsutil ls gs://cclebams/wes/Getz_IBM_DMX_CellLines_12samples_deepcov_01302018/RP-1561/Exome/RVH421SKINFV1/v1/RVH421SKINFV1.bam

In [ ]:
u = set(ccle_refsamples[ccle_refsamples.datatype=="wes"].legacy_bam_filepath)- set([a for a in i if a.endswith('.bam')])

In [ ]:
sizes = [gcp.extractSize(a) for a in gcp.lsFiles(set([a for a in i if a.endswith('.bam')]) - set(ccle_refsamples[ccle_refsamples.datatype=="wes"].legacy_bam_filepath), "-l")]

In [ ]:
sizes

In [ ]:
torm = []
tofind = []
for val in sizes:
    a = ccle_refsamples[ccle_refsamples.legacy_size==val[1]]
    if len(a)>0:
        torm.append(val[0])
        continue
    a = ccle_refsamples[ccle_refsamples['size']==val[1]]
    if len(a)>0:
        torm.append(val[0])
        continue
    tofind.append(val[0])

In [ ]:
! gsutil ls -l gs://cclebams/wes/RP-1500/Exome/ESO51_1/current/ESO51_1.bam

In [ ]:
! gsutil ls -l gs://cclebams/wes/CDS-CmkOx5.bam

In [ ]:
! gsutil ls -l gs://cclebams/wes/Getz_IBM_CellLine_WES_April2019/RP-1561/Exome/BPH1/v1/BPH1.bam

## PART 3

In [ ]:
## missing legacy for rna/wes
ccle_refsamples[ccle_refsamples.datatype.isin(['rna','wes'])& ccle_refsamples.legacy_bam_filepath.isna()]

In [ ]:
ccle_refsamples[ccle_refsamples.datatype.isin(['rna','wes'])& ccle_refsamples.legacy_bai_filepath.isna()]

In [ ]:
## wrong filepath legacy for rna/wes
miss_rwbi = gcp.exists(ccle_refsamples[ccle_refsamples.datatype.isin(['rna','wes'])].legacy_bai_filepath.tolist())

In [ ]:
miss_rw = gcp.exists(ccle_refsamples[ccle_refsamples.datatype.isin(['rna','wes'])].legacy_bam_filepath.tolist())

In [ ]:
## which ones can we get from remapping
remapable = ccle_refsamples[ccle_refsamples.legacy_bam_filepath.isin(miss_rw[1]) & ~ccle_refsamples.internal_bam_filepath.isna()]
len(remapable)

In [ ]:
remapable[remapable.datatype=='wes']

In [ ]:
## remapping
weswm = dm.WorkspaceManager("broad-firecloud-ccle/DepMap_WES_CN_hg38")

#weswm.update_sample_set("toremap", remapable.index.tolist())

In [ ]:
a = weswm.create_submission('BamToUnmappedRGBams_MC', "toremap", "sample_set", "this.samples")

In [ ]:
a = weswm.create_submission('Generate_uBAM_File_List', "toremap", "sample_set", "this.samples")

In [ ]:
a = weswm.create_submission('Realign_WES_GATK4_hg19', "toremap", "sample_set", "this.samples")

In [ ]:
## the ones we can't
nonremapable = ccle_refsamples[ccle_refsamples.legacy_bam_filepath.isin(miss_rw[1]) & ccle_refsamples.internal_bam_filepath.isna()]
nonremapable

In [ ]:
## indexing
a, missrwbi = gcp.exists(ccle_refsamples[~ccle_refsamples.legacy_bai_filepath.isna()].legacy_bai_filepath.tolist())

In [ ]:
a, missrw = gcp.exists(ccle_refsamples[~ccle_refsamples.legacy_bam_filepath.isna()].legacy_bam_filepath.tolist())

In [ ]:
missrwbirwbi

In [ ]:
bais = await seq.indexBams(bams=ccle_refsamples[(~ccle_refsamples.legacy_bai_filepath.isna()) |(ccle_refsamples.legacy_bai_filepath.isin(missrwbi))].legacy_bam_filepath, cores=12)

In [ ]:
bais = ! gsutil ls gs://cclebams/**.bai

In [ ]:
ccle_refsamples.loc[ccle_refsamples[((~ccle_refsamples.legacy_bam_filepath.isna()) &(ccle_refsamples.legacy_bai_filepath.isna())) |(ccle_refsamples.legacy_bai_filepath.isin(missrwbi))].index, 'legacy_bai_filepath'] = [i[:-4]+".bai" if i[:-4]+".bai" in bais else None for i in ccle_refsamples[((~ccle_refsamples.legacy_bam_filepath.isna()) &(ccle_refsamples.legacy_bai_filepath.isna())) |(ccle_refsamples.legacy_bai_filepath.isin(missrwbi))].legacy_bam_filepath]

In [ ]:
gcp.rmFiles(set(bais) - (set(ccle_refsamples.internal_bai_filepath) | set(ccle_refsamples.legacy_bai_filepath)), dryrun=False)

In [ ]:
len(set(ccle_refsamples.internal_bai_filepath) | set(ccle_refsamples.legacy_bai_filepath))

In [ ]:
ccle_refsamples[((~ccle_refsamples.legacy_bam_filepath.isna()) &(ccle_refsamples.legacy_bai_filepath.isna())) |(ccle_refsamples.legacy_bai_filepath.isin(missrwbi))].legacy_bam_filepath

In [ ]:
a = await seq.indexBams(ccle_refsamples[((~ccle_refsamples.legacy_bam_filepath.isna()) &(ccle_refsamples.legacy_bai_filepath.isna())) |(ccle_refsamples.legacy_bai_filepath.isin(missrwbi))].legacy_bam_filepath, cores=8)

In [ ]:
ccle_refsamples[((~ccle_refsamples.legacy_bam_filepath.isna()) &(ccle_refsamples.legacy_bai_filepath.isna())) |(ccle_refsamples.legacy_bai_filepath.isin(missrwbi))].legacy_bam_filepath

In [ ]:
## recompute size/ hashes for rna/wes legacy
ccle_refsamples.loc[ccle_refsamples[~ccle_refsamples.legacy_bam_filepath.isna() & ~ccle_refsamples.legacy_bam_filepath.isin(a[1])].index, "legacy_crc32c_hash"] = [gcp.extractHash(i) for i in gcp.lsFiles(ccle_refsamples[~ccle_refsamples.legacy_bam_filepath.isna() & ~ccle_refsamples.legacy_bam_filepath.isin(a[1])].legacy_bam_filepath, "-L")]
ccle_refsamples.loc[ccle_refsamples[~ccle_refsamples.legacy_bam_filepath.isna() & ~ccle_refsamples.legacy_bam_filepath.isin(a[1])].index, "legacy_size"] = [gcp.extractSize(i)[1] for i in gcp.lsFiles(ccle_refsamples[~ccle_refsamples.legacy_bam_filepath.isna() & ~ccle_refsamples.legacy_bam_filepath.isin(a[1])].legacy_bam_filepath, "-l")]

In [ ]:
## missing bam filepath 
missing = ccle_refsamples[ccle_refsamples.internal_bam_filepath.isna()]
missing

In [ ]:
## missing bai filepath
missing = ccle_refsamples[ccle_refsamples.internal_bai_filepath.isna()]
missing

In [ ]:
## wrong bam filepath
wrongfilepath = gcp.exists(ccle_refsamples[~ccle_refsamples.internal_bam_filepath.isna()].internal_bam_filepath.tolist())

In [ ]:
wrongfilepath

In [ ]:
## recompute size/hashes for bam filepath
ccle_refsamples.loc[ccle_refsamples[~ccle_refsamples.internal_bam_filepath.isna()].index, "crc32c_hash"] = [gcp.extractHash(i) for i in gcp.lsFiles(ccle_refsamples[~ccle_refsamples.internal_bam_filepath.isna()].internal_bam_filepath, "-L")]
ccle_refsamples.loc[ccle_refsamples[~ccle_refsamples.internal_bam_filepath.isna()].index, "size"] = [gcp.extractSize(i)[1] for i in gcp.lsFiles(ccle_refsamples[~ccle_refsamples.internal_bam_filepath.isna()].internal_bam_filepath, "-l")]
ccle_refsamples.loc[ccle_refsamples[~ccle_refsamples.internal_bam_filepath.isna()].index, "md5_hash"] = [gcp.extractHash(i, "md5") for i in gcp.lsFiles(ccle_refsamples[~ccle_refsamples.internal_bam_filepath.isna()].internal_bam_filepath, "-L")]

In [ ]:
## duplicate size accross legacy/internal 
h.dups(ccle_refsamples['legacy_size'].tolist())

In [ ]:
ccle_refsamples[ccle_refsamples['size'].isin([10555223974.0, 5176380746.0, 6707901359.0, 4634241136.0, 8984432599.0])].internal_bam_filepath

In [ ]:
remove = ['CDS-T5BcdG','CDS-gJupgp',"CDS-phR2eo","CDS-GuKG2u", "CDS-gLz8Kz",        "CDS-NUlX3d","CDS-b5ElTm","CDS-ljFuDX", "CDS-VS9XDY", "CDS-bb2V33", "CDS-lIXOWR","CDS-k9Qfva","CDS-9TDRpv","CDS-pBashm","CDS-6i6dRP","CDS-yu1s5X","CDS-g2J7MD","CDS-oRM8DN","CDS-2jBQ8n","CDS-c51IFr","CDS-59uKc2","CDS-363TYH","CDS-IQuj9W","CDS-R22IHj","CDS-up4Vo5","CDS-jSUD4f","CDS-RLVrVE","CDS-5x4qLj","CDS-6YFjST","CDS-2HO10g","CDS-vrqu12","CDS-pIy1EQ"]

In [ ]:
ccle_refsamples = ccle_refsamples[~ccle_refsamples.index.isin(['CDS-BNyMCM'])]

In [ ]:
,

In [ ]:
dm.WorkspaceManager("broad-firecloud-ccle/DepMap_WES_CN_hg38").delete_sample(set(["CDS-BNyMCM"]+remove) & set(dm.WorkspaceManager("broad-firecloud-ccle/DepMap_WES_CN_hg38").get_samples()))

In [ ]:
terra.removeSamples("broad-firecloud-ccle/DepMap_WES_CN_hg38",["CDS-BNyMCM"]+remove)

In [ ]:
ccle_refsamples[ccle_refsamples['legacy_size'].isin([19734437052.0])][['arxspan_id','version',"datatype","size","legacy_size"]].sort_values(by="legacy_size").iloc[:40]

In [ ]:
for val in h.dups(list(set(ccle_refsamples['size']))+list(set(ccle_refsamples['legacy_size']))):
    print(ccle_refsamples[(ccle_refsamples['size']==val)|(ccle_refsamples['legacy_size']==val)][['arxspan_id','version',"datatype","size","legacy_size"]])

In [ ]:
## missingbam index
wrongfilepath = gcp.exists(ccle_refsamples[~ccle_refsamples.internal_bai_filepath.isna()].internal_bai_filepath.tolist())

In [ ]:
## bam index non matching anything
bais= await seq.indexBams(bams= ccle_refsamples[ccle_refsamples.internal_bai_filepath.isin(wrongfilepath[1])].internal_bam_filepath, bucketpath=None, cores=10)

In [ ]:
for k,val in bais.items():
    ccle_refsamples.loc[ccle_refsamples[ccle_refsamples.internal_bam_filepath == k].index, 'internal_bai_filepath'] = val

In [ ]:
dfToSheet(ccle_refsamples,'ccle sample tracker', secret=creds)

In [ ]:
i  = ! gsutil ls gs://cclebams/**

In [ ]:
## bam non matching anything in the sample tracker
nonmatch = set([a for a in i if a.endswith('.bam')]) - set(ccle_refsamples.internal_bam_filepath.tolist()+ ccle_refsamples.legacy_bam_filepath.tolist())
nonmatch

In [ ]:
## bam that match something in size and can be deleted or can replace the other one
sizze=[gcp.extractSize(i) for i in gcp.lsFiles(nonmatch, '-l')]

In [ ]:
sizze

In [ ]:
miss_rw

In [ ]:
found=[]
torm=[]
remaining = set()
for val in sizze:
    res= ccle_refsamples[(ccle_refsamples['legacy_size']==val[1])]
    if len(res) >0:
        if res['legacy_bam_filepath'].values[0] in miss_rw[1]:
            print("found!-----------")
            ccle_refsamples.loc[res.index, 'legacy_bam_filepath'] = val[0]
            found.append(res.index[0])
        else:
            torm.append(val[0])
    else:
        remaining.add(val[0])

In [ ]:
found

In [ ]:
torm = ['gs://cclebams/wes/CDS-eGQYXr.bam',
 'gs://cclebams/wes/CDS-Fz0HXE.bam',
 'gs://cclebams/wes/CDS-4S6juQ.bam',
 'gs://cclebams/wes/CDS-UL1jLm.bam',
 'gs://cclebams/rna/UPCISCC111.bam',
 'gs://cclebams/wes/CDS-gKIdjs.bam',
 'gs://cclebams/rna/TERA2.bam',
 'gs://cclebams/wes/CDS-Hx6zuD.bam',
 'gs://cclebams/wes/CDS-Xgu4mi.bam',
 'gs://cclebams/wes/CDS-kt2Gne.bam',
 'gs://cclebams/wes/CDS-SvzhGj.bam',
 'gs://cclebams/wes/CDS-Pku96X.bam',
 'gs://cclebams/wes/CDS-leyYAD.bam',
 'gs://cclebams/wes/CDS-R6ehaT.bam',
 'gs://cclebams/wes/CDS-cKMeDY.bam',
 'gs://cclebams/wes/CDS-I7bMcd.bam',
 'gs://cclebams/wes/CDS-GnBdHN.bam',
 'gs://cclebams/wes/CDS-PHI8VT.bam',
 'gs://cclebams/wes/CDS-Y27yfi.bam',
 'gs://cclebams/wes/CDS-sieIuO.bam',
 'gs://cclebams/rna/MM253.bam',
 'gs://cclebams/wes/CDS-3mvYnW.bam',
 'gs://cclebams/rna/PEDS117.bam',
 'gs://cclebams/wes/CDS-AOWMF3.bam',
 'gs://cclebams/wes/CDS-8z476r.bam',
 'gs://cclebams/wes/CDS-qaOoHQ.bam',
 'gs://cclebams/wes/CDS-xCyamv.bam',
 'gs://cclebams/wes/CDS-6PZKz8.bam',
 'gs://cclebams/wes/CDS-N83rwD.bam',
 'gs://cclebams/wes/CDS-5bQzF2.bam',
 'gs://cclebams/wes/CDS-3pZIvU.bam',
 'gs://cclebams/wes/CDS-R3txwY.bam',
 'gs://cclebams/wes/CDS-O8dfj7.bam',
 'gs://cclebams/wes/CDS-D6mIfI.bam',
 'gs://cclebams/wes/CDS-9sg0Pm.bam',
 'gs://cclebams/wes/CDS-C7o0op.bam',
 'gs://cclebams/wes/CDS-aDUHcI.bam',
 'gs://cclebams/wes/CDS-YnodyM.bam',
 'gs://cclebams/wes/CDS-s7pOQR.bam',
 'gs://cclebams/wes/CDS-V6Kk5q.bam',
 'gs://cclebams/wes/CDS-WP95Oi.bam',
 'gs://cclebams/wes/CDS-HkZUmY.bam',
 'gs://cclebams/wes/CDS-rVAuin.bam',
 'gs://cclebams/wes/CDS-LifesX.bam',
 'gs://cclebams/wes/CDS-NBnCDl.bam',
 'gs://cclebams/wes/CDS-J6kDsZ.bam',
 'gs://cclebams/wes/CDS-LOW19e.bam',
 'gs://cclebams/wes/CDS-h4mOdz.bam',
 'gs://cclebams/wes/CDS-9u5DMn.bam',
 'gs://cclebams/wes/CDS-rQMY3G.bam',
 'gs://cclebams/wes/CDS-9M8GNS.bam',
 'gs://cclebams/wes/CDS-wSV3OM.bam',
 'gs://cclebams/wes/CDS-4BrJr7.bam',
 'gs://cclebams/wes/CDS-sbwn0P.bam',
 'gs://cclebams/wes/CDS-WAPQGk.bam',
 'gs://cclebams/wes/CDS-tORJC8.bam',
 'gs://cclebams/rna/SNU2372.bam',
 'gs://cclebams/wes/CDS-CZstO2.bam',
 'gs://cclebams/wes/CDS-bdb5iE.bam',
 'gs://cclebams/rna/SUM190PT.bam',
 'gs://cclebams/wes/CDS-opnGD7.bam',
 'gs://cclebams/wes/CDS-c2Sowd.bam',
 'gs://cclebams/wes/CDS-NPG23x.bam',
 'gs://cclebams/wes/CDS-w7i5l7.bam',
 'gs://cclebams/wes/CDS-iRstNJ.bam',
 'gs://cclebams/rna/HEMCSS.bam',
 'gs://cclebams/wes/CDS-ZGlgTf.bam',
 'gs://cclebams/wes/CDS-ohjYlg.bam',
 'gs://cclebams/wes/CDS-RWYJ02.bam',
 'gs://cclebams/wes/CDS-ogUnWk.bam',
 'gs://cclebams/wes/CDS-ktRRkc.bam',
 'gs://cclebams/wes/CDS-Qbfoau.bam',
 'gs://cclebams/wes/CDS-m49nRz.bam',
 'gs://cclebams/wes/CDS-P79y6z.bam',
 'gs://cclebams/wes/CDS-lTogDX.bam',
 'gs://cclebams/wes/CDS-iX8vqU.bam',
 'gs://cclebams/wes/CDS-5hbofu.bam',
 'gs://cclebams/wes/CDS-nTW67d.bam',
 'gs://cclebams/wes/CDS-ldrQm3.bam',
 'gs://cclebams/wes/CDS-xI8ZAZ.bam',
 'gs://cclebams/rna/VAL.bam',
 'gs://cclebams/wes/CDS-UvBswk.bam',
 'gs://cclebams/wes/CDS-HjGCvC.bam',
 'gs://cclebams/rna/UPCISCC072.bam',
 'gs://cclebams/wes/CDS-YYd4ww.bam',
 'gs://cclebams/wes/CDS-IzeN7a.bam',
 'gs://cclebams/wes/CDS-2JxT1P.bam',
 'gs://cclebams/wes/CDS-rLadW7.bam',
 'gs://cclebams/wes/CDS-ctVpqU.bam',
 'gs://cclebams/wes/CDS-xIv1KJ.bam',
 'gs://cclebams/wes/CDS-ragHOy.bam',
 'gs://cclebams/wes/CDS-4ZOQQF.bam',
 'gs://cclebams/wes/CDS-6xyqy9.bam',
 'gs://cclebams/wes/CDS-Iu8c04.bam',
 'gs://cclebams/wes/CDS-wpXVQk.bam',
 'gs://cclebams/wes/CDS-soTPPi.bam',
 'gs://cclebams/wes/CDS-5ViPeM.bam',
 'gs://cclebams/wes/CDS-EzZEgz.bam',
 'gs://cclebams/wes/CDS-gCSYjV.bam',
 'gs://cclebams/rna/NOS1.bam',
 'gs://cclebams/rna/ONDA10.bam',
 'gs://cclebams/wes/CDS-Ww1LC7.bam',
 'gs://cclebams/wes/CDS-BRxHbu.bam',
 'gs://cclebams/rna/MM466.bam',
 'gs://cclebams/wes/CDS-OxQgBw.bam',
 'gs://cclebams/wes/CDS-RFBAY6.bam',
 'gs://cclebams/wes/CDS-rQIdNN.bam',
 'gs://cclebams/wes/CDS-4l9BUT.bam',
 'gs://cclebams/wes/CDS-Bojgi7.bam',
 'gs://cclebams/wes/CDS-6Fc0S5.bam',
 'gs://cclebams/wes/CDS-DZMoWW.bam',
 'gs://cclebams/wes/CDS-HEoDm7.bam',
 'gs://cclebams/wes/CDS-IGOgCK.bam',
 'gs://cclebams/wes/CDS-AjRIMt.bam',
 'gs://cclebams/wes/CDS-G1sVsw.bam',
 'gs://cclebams/rna/NCCIT.bam',
 'gs://cclebams/wes/CDS-8mpXJa.bam',
 'gs://cclebams/wes/CDS-mtMTts.bam',
 'gs://cclebams/wes/CDS-75psAH.bam',
 'gs://cclebams/wes/CDS-LnV7QY.bam',
 'gs://cclebams/wes/CDS-ydPJEM.bam',
 'gs://cclebams/wes/CDS-tPR3fn.bam',
 'gs://cclebams/wes/CDS-PYWxsh.bam',
 'gs://cclebams/wes/CDS-wWwBMZ.bam',
 'gs://cclebams/wes/CDS-frzvLf.bam',
 'gs://cclebams/wes/CDS-wlTAAF.bam',
 'gs://cclebams/wes/CDS-UVxUrF.bam',
 'gs://cclebams/wes/CDS-x21VqU.bam',
 'gs://cclebams/wes/CDS-tYXity.bam',
 'gs://cclebams/rna/RO82W1.bam',
 'gs://cclebams/wes/CDS-Md89va.bam',
 'gs://cclebams/wes/CDS-mazUYU.bam',
 'gs://cclebams/wes/CDS-wbPtTZ.bam',
 'gs://cclebams/wes/CDS-gsqqAz.bam',
 'gs://cclebams/wes/CDS-ik526H.bam',
 'gs://cclebams/wes/CDS-83LhEq.bam',
 'gs://cclebams/wes/CDS-MhXQX3.bam',
 'gs://cclebams/wes/CDS-Awmxa5.bam',
 'gs://cclebams/wes/CDS-z8Bvmk.bam',
 'gs://cclebams/wes/CDS-HOVBCg.bam',
 'gs://cclebams/wes/CDS-0qPmaJ.bam',
 'gs://cclebams/wes/CDS-Pkk9e2.bam',
 'gs://cclebams/wes/CDS-6EyvRQ.bam',
 'gs://cclebams/wes/CDS-nYIBWR.bam',
 'gs://cclebams/wes/CDS-NXnWiI.bam',
 'gs://cclebams/wes/CDS-8aHSii.bam',
 'gs://cclebams/wes/CDS-Eh7ost.bam',
 'gs://cclebams/wes/CDS-uGZguG.bam',
 'gs://cclebams/wes/CDS-CRPZeK.bam',
 'gs://cclebams/wes/CDS-n7Fqfe.bam',
 'gs://cclebams/wes/CDS-1PXzlf.bam',
 'gs://cclebams/wes/CDS-b9sdh9.bam',
 'gs://cclebams/wes/CDS-LNTGnh.bam',
 'gs://cclebams/wes/CDS-M1sAGX.bam',
 'gs://cclebams/wes/CDS-dNVjOc.bam',
 'gs://cclebams/wes/CDS-fLsYaB.bam',
 'gs://cclebams/wes/CDS-loy9vi.bam',
 'gs://cclebams/wes/CDS-nOKbmw.bam',
 'gs://cclebams/wes/CDS-SO3AhH.bam',
 'gs://cclebams/wes/CDS-r5Ym7C.bam',
 'gs://cclebams/rna/MP41.bam',
 'gs://cclebams/rna/OZ.bam',
 'gs://cclebams/wes/CDS-Hv0i3y.bam',
 'gs://cclebams/wes/CDS-Hw6KuA.bam',
 'gs://cclebams/rna/TANOUE.bam',
 'gs://cclebams/rna/U2973.bam',
 'gs://cclebams/wes/CDS-allHxr.bam',
 'gs://cclebams/wes/CDS-TDblpN.bam',
 'gs://cclebams/wes/CDS-3jIdRa.bam',
 'gs://cclebams/wes/CDS-ck9vpG.bam',
 'gs://cclebams/wes/CDS-EpURcL.bam',
 'gs://cclebams/rna/TL1.bam',
 'gs://cclebams/wes/CDS-2Q2Kia.bam',
 'gs://cclebams/wes/CDS-LUm1Vn.bam',
 'gs://cclebams/rna/rna/DepMap_CellLine_RNAseq_Feb2020/RP-1561/RNA/CHL-1/v2/CHL-1.bam',
 'gs://cclebams/wes/CDS-96DdrP.bam',
 'gs://cclebams/wes/CDS-TSDUCK.bam',
 'gs://cclebams/wes/CDS-ZMsoXe.bam',
 'gs://cclebams/wes/CDS-8yHnJv.bam',
 'gs://cclebams/wes/CDS-7JWzyA.bam',
 'gs://cclebams/wes/CDS-bPT1F0.bam',
 'gs://cclebams/wes/CDS-ycD9px.bam',
 'gs://cclebams/wes/CDS-Ql8GJZ.bam',
 'gs://cclebams/wes/CDS-g0KUGN.bam',
 'gs://cclebams/rna/TY82.bam',
 'gs://cclebams/rna/WSUFSCCL.bam',
 'gs://cclebams/wes/CDS-2lAFkD.bam',
 'gs://cclebams/rna/MM485.bam',
 'gs://cclebams/wes/CDS-XJDBDj.bam',
 'gs://cclebams/wes/CDS-PyELSk.bam',
 'gs://cclebams/wes/CDS-kAARUi.bam',
 'gs://cclebams/wes/CDS-rUs3FP.bam',
 'gs://cclebams/wes/CDS-Uru0Mh.bam',
 'gs://cclebams/wes/CDS-aXqwpM.bam',
 'gs://cclebams/wes/CDS-l1OClV.bam',
 'gs://cclebams/wes/CDS-lSpYo6.bam',
 'gs://cclebams/wes/CDS-5IcijG.bam',
 'gs://cclebams/wes/CDS-RnsUHX.bam',
 'gs://cclebams/wes/CDS-9zidMf.bam',
 'gs://cclebams/rna/KKU055.bam',
 'gs://cclebams/wes/CDS-XqaEOX.bam',
 'gs://cclebams/rna/NH12.bam',
 'gs://cclebams/wes/CDS-aGMcvr.bam',
 'gs://cclebams/wes/CDS-QtTdY6.bam',
 'gs://cclebams/wes/CDS-8sQWae.bam',
 'gs://cclebams/wes/CDS-jHqXGP.bam',
 'gs://cclebams/wes/CDS-LVeuLY.bam',
 'gs://cclebams/wes/CDS-2xSJmZ.bam',
 'gs://cclebams/rna/OSC20.bam',
 'gs://cclebams/wes/CDS-5PXB9Y.bam',
 'gs://cclebams/wes/CDS-u1AlUI.bam',
 'gs://cclebams/wes/9505BIK.bam',
 'gs://cclebams/wes/CDS-mGHY2S.bam',
 'gs://cclebams/wes/CDS-49xzNU.bam',
 'gs://cclebams/wes/CDS-3DHwSX.bam',
 'gs://cclebams/wes/CDS-V2ZEuP.bam',
 'gs://cclebams/wes/CDS-VCuHjJ.bam',
 'gs://cclebams/wes/CDS-cmV75B.bam',
 'gs://cclebams/wes/CDS-OnIxUL.bam',
 'gs://cclebams/wes/CDS-HoW111.bam',
 'gs://cclebams/wes/CDS-cMvnjL.bam',
 'gs://cclebams/wes/CDS-dJqQ4g.bam',
 'gs://cclebams/wes/CDS-Ty3mgt.bam',
 'gs://cclebams/wes/CDS-eZg4P8.bam',
 'gs://cclebams/wes/CDS-9JpX07.bam',
 'gs://cclebams/rna/RPMI2650.bam',
 'gs://cclebams/wes/CDS-3FueNQ.bam',
 'gs://cclebams/wes/CDS-O1ShTQ.bam',
 'gs://cclebams/wes/CDS-RxQhcq.bam',
 'gs://cclebams/rna/HDMB03.bam',
 'gs://cclebams/wes/CDS-3VNhFC.bam',
 'gs://cclebams/wes/CDS-sCWLGL.bam',
 'gs://cclebams/wes/CDS-x7srFK.bam',
 'gs://cclebams/wes/CDS-Twv1kD.bam',
 'gs://cclebams/rna/HAC2.bam',
 'gs://cclebams/wes/CDS-qIc5x3.bam',
 'gs://cclebams/wes/CDS-AJMYsd.bam',
 'gs://cclebams/wes/CDS-hOI086.bam',
 'gs://cclebams/wes/CDS-5H2go6.bam',
 'gs://cclebams/wes/CDS-zwAn7G.bam',
 'gs://cclebams/rna/LO68.bam',
 'gs://cclebams/wes/CDS-294bk6.bam',
 'gs://cclebams/wes/CDS-ziEOXJ.bam',
 'gs://cclebams/wes/CDS-VBr00g.bam',
 'gs://cclebams/rna/ULA.bam',
 'gs://cclebams/wes/CDS-dWHWU3.bam',
 'gs://cclebams/rna/MMNK1.bam',
 'gs://cclebams/wes/CDS-H4hPhD.bam',
 'gs://cclebams/wes/CDS-cBOy2Z.bam',
 'gs://cclebams/wes/CDS-NZsio7.bam',
 'gs://cclebams/wes/CDS-5LNjjI.bam',
 'gs://cclebams/wes/CDS-JvOeJK.bam',
 'gs://cclebams/wes/CDS-xKNh7Q.bam',
 'gs://cclebams/wes/CDS-2hGt1N.bam',
 'gs://cclebams/wes/CDS-tgnRyK.bam',
 'gs://cclebams/wes/CDS-Eo5oAR.bam',
 'gs://cclebams/wes/CDS-T10Uph.bam',
 'gs://cclebams/wes/CDS-QE7bdY.bam',
 'gs://cclebams/wes/CDS-1uWUTi.bam']
torm

In [ ]:
for val in sizze:
    res= ccle_refsamples[(ccle_refsamples['size']==val[1])]
    if len(res) >0:
        print(val[0], res['internal_bam_filepath'])
        torm.append(val[0])
        remaining.remove(val[0])

In [ ]:
## removing
a = gcp.rmFiles(torm, dryrun=False)

In [ ]:
## preventing remapping of found ones
weswm.update_sample_set("toremap", set(remapable.index) - set(found))

In [ ]:
# loading the remapped version to ccle sample tracker
ss = weswm.get_sample_sets()

In [ ]:
sam = weswm.get_samples().loc[ss.loc['toremap'].samples]

In [ ]:
e = []
for i in sam["readgroup_ubams"].values.tolist():
    if i is not np.nan:
        e.extend(i)

In [ ]:
gcp.rmFiles(e)

In [ ]:
newloc = terra.changeGSlocation("broad-firecloud-ccle/DepMap_WES_CN_hg38", newgs="gs://cclebams/wes/", onlycol=['hg38_analysis_ready_bam', 'hg38_analysis_ready_bam_index'], entity='sample', keeppath=False, dry_run = False, onlysamples=ss.loc['toremap'].samples)

In [ ]:
newloc = newloc[0].applymap(lambda v: v.replace('.hg38.', '.hg19.'))

In [ ]:
h.parrun(['gsutil mv '+val+' '+val.replace('.hg38.', '.hg19.') for val in newloc[0].values.ravel()], 10)

In [ ]:
newloc[['hg38_analysis_ready_bam',"hg38_analysis_ready_bam_index"]] =  ccle_refsamples.loc[newloc.index, ['internal_bam_filepath','internal_bai_filepath']]

In [ ]:
ccle_refsamples.loc[newloc.index, ['legacy_bam_filepath','legacy_bai_filepath']] = newloc.values

In [ ]:
weswm.update_sample_attributes(newloc)

In [ ]:
ccle_refsamples.loc[newloc.index, ['legacy_bam_filepath','legacy_bai_filepath']]

In [ ]:
## are they very close to a related sample? (yes? drop)

In [ ]:
## can we find a related CDS id? (yes? find out if close else drop)
ccle_refsamples[ccle_refsamples.index.isin([i.split('/')[-1].split('.')[0] for i in remaining if "CDS" in i])]

In [ ]:
## do we have this cell line already? (investigate, either add or drop)
sizze = [gcp.extractSize(u) for u in gcp.lsFiles([i for i in remaining if "CDS" not in i],'-l')]

In [ ]:
for val in sizze:
    print('------------')
    print(val)
    a = h.closest(ccle_refsamples['size'], val[1])
    print(a)
    print(ccle_refsamples[ccle_refsamples['size'] == a][['stripped_cell_name','arxspan_id']])

In [ ]:
gs://cclebams/wes/RP-1500/Exome/ESO26_1/current/ESO26_1.bam'

In [ ]:
for val in sizze:
    print('------------')
    print(val)
    a = h.closest(ccle_refsamples[~ccle_refsamples['legacy_size'].isna()]['legacy_size'], val[1])
    print(a)
    print(ccle_refsamples[ccle_refsamples['legacy_size'] == a][['stripped_cell_name','arxspan_id']])

In [ ]:
## match md5 to bam files
md5s = [u for u in i if u.endswith('.md5')]
for val in md5s:
    rres = ccle_refsamples[(ccle_refsamples.internal_bam_filepath==val[:-4])&(ccle_refsamples.md5_hash.isna())]
    if len(rres)>0:
        print(rres)

In [ ]:
## delete md5s
gcp.rmFiles(md5s, dryrun=False)

In [ ]:
## missing tissue annot
set(ccle_refsamples[ccle_refsamples['primary_site'].isna() | ccle_refsamples['primary_disease'].isna() | ccle_refsamples['subtype'].isna() | ccle_refsamples['origin'].isna()].arxspan_id)

In [ ]:
## missing name/arxspan
ccle_refsamples[ccle_refsamples['arxspan_id'].isna() | ccle_refsamples['stripped_cell_name'].isna()]

In [ ]:
# clean annots:
set(ccle_refsamples.primary_disease)

In [ ]:
set(ccle_refsamples.primary_site)

In [ ]:
set(ccle_refsamples.age)

In [ ]:
ccle_refsamples[ccle_refsamples.age.isna()]

In [ ]:
set(ccle_refsamples.subtype)

In [ ]:
set(ccle_refsamples.origin)

In [ ]:
set(ccle_refsamples[ccle_refsamples.source.isna()].arxspan_id)

In [ ]:
ccle_refsamples[ccle_refsamples.sex.isna()]

In [ ]:
set(ccle_refsamples.baits)

In [ ]:
ccle_refsamples.loc[k, upd]

In [ ]:
k

In [ ]:
upd = ['participant_id', 'cellosaurus_id', 'parent_cell_line', 'sex',
       'matched_normal', 'age', 'primary_site', 'primary_disease', 'subtype',
       'subsubtype', 'origin']
for k in ["CDS-yYeDem",
            "CDS-OT217E",
            "CDS-a8BCnW",
            "CDS-zeFugQ",
            "CDS-GOI63U",
            "CDS-ldvG6N",
            "CDS-LFK9QA",
            "CDS-SdYhEt",
            "CDS-gyyVs5",
            "CDS-IgYH22",
            "CDS-PoGjEx",
            "CDS-J80Eb1",
            "CDS-Hb7hHv",
            "CDS-3VSkPT",
            "CDS-2FgC1U"]:
    res = ccle_refsamples[(ccle_refsamples.arxspan_id==ccle_refsamples.loc[k,"arxspan_id"]) &(~ccle_refsamples.age.isna()) & (ccle_refsamples.index!=k)]
    if len(res)>0:
        ccle_refsamples.loc[k, upd] = res.iloc[0][upd].values
    else:
        print(k)

In [ ]:
upd = ['participant_id', 'cellosaurus_id', 'parent_cell_line', 'sex',
       'matched_normal', 'age', 'primary_site', 'primary_disease', 'subtype',
       'subsubtype', 'origin']
for k, v in ccle_refsamples[ccle_refsamples['primary_site'].isna() | ccle_refsamples['primary_disease'].isna() | ccle_refsamples['subtype'].isna() | ccle_refsamples['origin'].isna() | ccle_refsamples['source'].isna()].iterrows():
    res = ccle_refsamples[(ccle_refsamples.arxspan_id==v.arxspan_id) & (ccle_refsamples.index!=k)]
    if len(res)>0:
        ccle_refsamples.loc[k, upd] = res.iloc[0][upd].values

In [ ]:
## missing tissue annot
len(set(ccle_refsamples[ccle_refsamples['primary_site'].isna() | ccle_refsamples['primary_disease'].isna() | ccle_refsamples['subtype'].isna() | ccle_refsamples['origin'].isna()].arxspan_id))

In [ ]:
ccle_refsamples[(ccle_refsamples.arxspan_id==val.arxspan_id) &(~ccle_refsamples.age.isna())].iloc[0][upd].values

In [ ]:
dfToSheet(ccle_refsamples,'ccle sample tracker', secret=creds)

## PART 4

In [ ]:
wm.disable_hound().update_sample_attributes(a)

In [ ]:
wm = dm.WorkspaceManager('broad-firecloud-ccle/DepMap_hg38_RNAseq')

In [ ]:
a = wm.get_samples()

In [ ]:
ss= h.dups(ccle_refsamples['size'].tolist()+ccle_refsamples['legacy_size'].tolist())

In [ ]:
issue = {val:[] for val in ss}
recom = []
for val in ss:
    for k,v in ccle_refsamples[(ccle_refsamples['size']==val)].iterrows():
        if v.size!= gcp.extractSize(gcp.lsFiles([v.internal_bam_filepath],"-l")[0]):
            recom.append(k)
        else:
            issue[val].append(k)
    for k,v in ccle_refsamples[(ccle_refsamples['legacy_size']==val)].iterrows():
        if v.legacy_size!= gcp.extractSize(gcp.lsFiles([v.legacy_bam_filepath],"-l")[0]):
            recom.append(k)    
        else:
            issue[val].append(k)

In [ ]:
e = gcp.lsFiles(a.legacy_bam_filepath.tolist(),"-L")

In [ ]:
wm.update_sample_attributes(a)

In [ ]:
for k, val in ccle_refsamples[(ccle_refsamples.datatype=='rna')&(~ccle_refsamples.internal_bam_filepath.str.contains("/rnasq_hg38",na=True))].internal_bam_filepath.iteritems():
    rn = "gs://cclebams/rna/"+'/'.join(val.split('/')[3:])
    print(rn)
    rni = rn[:-1]+'i'
    vali = val[:-1]+"i"
    #! gsutil cp $val $rn
    #! gsutil cp $vali $rni
    ccle_refsamples.loc[k, "internal_bam_filepath"] = rn
    ccle_refsamples.loc[k, "internal_bai_filepath"] = rni
    ccle_refsamples.loc[k, "size"] = gcp.extractSize(gcp.lsFiles([rn], '-l')[0],)[1]
    ccle_refsamples.loc[k, "crc32c_hash"] = gcp.extractHash(gcp.lsFiles([rn], '-L')[0])[1]
    ccle_refsamples.loc[k, "sequencing_date"]= seq.getBamDate([rn])[0]
    print(rn)
    break

In [ ]:
ccle_refsamples.loc[a[a.legacy_bam_filepath.isna()].index].internal_bam_filepath

In [ ]:
ccle_refsamples.loc[ccle_refsamples[ccle_refsamples.datatype.isin(['rna'])].index.tolist(),'size']

In [ ]:
ccle_refsamples.loc[ccle_refsamples[ccle_refsamples.datatype.isin(['rna'])].index.tolist(),'size'] = [gcp.extractSize(i)[1] for i in gcp.lsFiles(ccle_refsamples[ccle_refsamples.datatype.isin(['rna'])]['internal_bam_filepath'].tolist(),'-l')]

ccle_refsamples.loc[ccle_refsamples[ccle_refsamples.datatype.isin(['rna'])].index.tolist(),'crc32c_hash'] = [gcp.extractHash(i) for i in gcp.lsFiles(ccle_refsamples[ccle_refsamples.datatype.isin(['rna'])]['internal_bam_filepath'].tolist(),'-L')]

In [ ]:
ccle_refsamples[ccle_refsamples.datatype.isin(['wgs'])][["internal_bam_filepath", "internal_bai_filepath", "md5_hash","legacy_bam_filepath","legacy_bai_filepath",'legacy_size', 'legacy_crc32c_hash']]